# Assignment # 3 ####

In [2]:
# Loading the dependencies for the model
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math

In [3]:
iris = load_iris()
X = iris.data
y = iris.target
y_labels = iris.target_names
X_labels = iris.feature_names

In [4]:
#4 features, 150 samples, 3 target values to predict
print("Column names - ",X_labels)
print("Shape of X - ",X.shape)
print("Target values - ",y_labels)

Column names -  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Shape of X -  (150, 4)
Target values -  ['setosa' 'versicolor' 'virginica']


In [36]:
#Putting all the data into a dataframe
iris = pd.DataFrame(X, columns=X_labels)
iris['Species'] = y
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [37]:
iris.shape

(150, 5)

In [38]:
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [39]:
iris.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [40]:
#Calculating class priors (simply the probability of class labels)
priors = np.unique(y, return_counts=True)[1] / len(y)
priors

array([0.33333333, 0.33333333, 0.33333333])

In [41]:
#Class wise mean for each feature
Species_means = iris.groupby(['Species']).mean().reset_index()
Species_means

,Species,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.006,3.428,1.462,0.246
1,1,5.936,2.770,4.260,1.326
2,2,6.588,2.974,5.552,2.026


In [55]:
#Class wise variance for each feature
Species_Var = iris.groupby(['Species']).var().reset_index()
Species_Var

,Species,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,0.124249,0.143690,0.030159,0.011106
1,1,0.266433,0.098469,0.220816,0.039106
2,2,0.404343,0.104004,0.304588,0.075433


In [56]:
#Class wise std for each feature
Species_std = iris.groupby(['Species']).std().reset_index()
Species_std

,Species,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,0.352490,0.379064,0.173664,0.105386
1,1,0.516171,0.313798,0.469911,0.197753
2,2,0.635880,0.322497,0.551895,0.274650


In [57]:
# Gaussian probability function vectorized to work on multidimensional arrays with broadcasting
def pdf(x, mean, std):
    return (1 / np.sqrt(2*np.pi*std**2)) * np.exp(-1*((x-mean)**2/(2*std**2)))
vectorized_pdf = np.vectorize(pdf)

In [76]:
##Prediction using First point [4.5,3.0,5.6,2.1]##
#Converting everything to numpy arrays 
mean_array = np.array(Species_means.iloc[:,1:])
std_array = np.array(Species_std.iloc[:,1:])
#Sample is the feature vector, whose label needs to be predicted
#Input Vector 
X1 = np.array([[4.5,3.0,5.6,2.1]])
print("X1.shape: {}".format(X1.shape))

X1.shape: (1, 4)


In [77]:
#Element wise application of the pdf(x,mean,std) function on broadcasted arrays
elementwise_pdf1 = vectorized_pdf(X1, mean_array, std_array)
elementwise_pdf1

array([[3.89604219e-034, 4.93034074e-013, 9.85229377e-224,
        1.83350725e-085],
       [1.90758805e-015, 3.45722573e-019, 2.97900952e-029,
        5.25539874e-024],
       [6.15248903e-010, 3.81797812e-018, 7.37455641e-021,
        2.30539266e-012]])

In [78]:
#Rowwise product of the above matrix
rowwise_product1 = np.product(elementwise_pdf1,axis=1)
rowwise_product1

array([0.00000000e+00, 1.03249960e-85, 3.99360485e-59])

In [79]:
#Elementwise multiplication with corresponding class probabilities (priors)
multiply_priors1 = np.multiply(priors,rowwise_product1)
multiply_priors1

array([0.00000000e+00, 3.44166534e-86, 1.33120162e-59])

In [80]:
#Argmax to predict class label
prediction1 = np.argmax(multiply_priors1)
prediction1

2

# The highest probability for [4.5,3.0,5.6,2.1] is of Iris Virginica. 

In [83]:
##Prediction using Second point [5.4,2.6,4.5,0.0]##
#Converting everything to numpy arrays 
mean_array = np.array(Species_means.iloc[:,1:])
std_array = np.array(Species_std.iloc[:,1:])
#Sample is the feature vector, whose label needs to be predicted
#Input Vector 
X2 = np.array([[5.4,2.6,4.5,0.0]])
print("X2.shape: {}".format(X2.shape))

X2.shape: (1, 4)


In [84]:
#Element wise application of the pdf(x,mean,std) function on broadcasted arrays
elementwise_pdf2 = vectorized_pdf(X2, mean_array, std_array)
elementwise_pdf2

array([[2.56847506e-049, 8.02227159e-010, 3.22274138e-144,
        3.76458557e+000],
       [2.56770327e-022, 2.01704638e-014, 8.35110848e-019,
        1.97831708e+000],
       [2.48263318e-014, 1.21604485e-013, 2.04443269e-013,
        1.39878312e+000]])

In [85]:
#Rowwise product of the above matrix
rowwise_product2 = np.product(elementwise_pdf2,axis=1)
rowwise_product2

array([2.49985801e-201, 8.55659046e-054, 8.63346921e-040])

In [86]:
#Elementwise multiplication with corresponding class probabilities (priors)
multiply_priors2 = np.multiply(priors,rowwise_product2)
multiply_priors2

array([8.33286004e-202, 2.85219682e-054, 2.87782307e-040])

In [87]:
#Argmax to predict class label
prediction2 = np.argmax(multiply_priors2)
prediction2

2

# Similarly, The highest probability for [5.4,2.6,4.5,0.0] is of Iris Virginica. 